# **Avaliação Alternativa 1**

In [1]:
import sys

sys.path.append('../')

In [2]:
import numpy as np
import pandas as pd
from src.utils.binary_multiply import binary_multiply
from src.utils.binary_to_decimal import binary_to_decimal

## Problema 1

<div style="text-align: center;">
    <img src="../src/images/bisection_method_graph.png" width="400">
</div>

**Teorema:** Seja $f(x)$ uma função contínua num intervalo $[a, b]$. Se $f(a)f(b) < 0$, então existe pelo menos um ponto $x = \alpha$, onde $\alpha \in (a, b)$, tal que $f(\alpha) = 0$.
1. $[a, b]$ é o intervalo inicial, $\epsilon$ é o erro objetivo, $k = 0$.
2. Faça $x =  \frac{a + b}{2}$
3. Se $f(a)f(x) < 0$, faça $b = x$. Caso contrário, faça $a = x$.
4. Se $b - a < \epsilon$, então escolha um $\overline x$ qualquer em $[a, b]$ e pare. Caso contrário $k = k + 1$ e volte ao passo 2.

In [3]:
def split_intervals(derivative, a, b):
    sample_points = np.linspace(a, b)
    derivative_values = [derivative(x) for x in sample_points]

    if all(d >= 0 for d in derivative_values) or all(d <= 0 for d in derivative_values):
        return [(a, b)]

    mid = (a + b) / 2

    left_intervals = split_intervals(derivative, a, mid)
    right_intervals = split_intervals(derivative, mid, b)

    return left_intervals + right_intervals

def bisection_method(function, derivative, step_size):
    intervals = []
    x_grid = np.arange(-101, 101, step_size)

    for x in x_grid:
        if (function(x) <= 0 and function(x + step_size) >= 0) or (function(x) >= 0 and function(x + step_size) <= 0):
            intervals.extend(split_intervals(derivative, x, x + step_size))

    roots = []

    for interval in intervals:
        print(f'Intervalo: {(float(interval[0]), float(interval[1]))}')
        a = interval[0]
        b = interval[1]
        epsilon = 0.001

        log = {'Iteração': [], 'x': [], 'f(x)': [], 'b - a': []}

        iteration = 1
        while b - a > epsilon:
            x = (a + b) / 2

            if function(a) * function(x) < 0:
                b = x
            else:
                a = x

            log['Iteração'].append(iteration)
            log['x'].append(x)
            log['f(x)'].append(function(x))
            log['b - a'].append(b - a)
  
            iteration += 1
        
        df = pd.DataFrame(log)
        display(df.set_index('Iteração'))

        roots.append(float((a + b) / 2))
        print()

    return roots

In [4]:
def function(x):
    return x ** 3 - 9 * x ** 2 + 3

def derivative(x):
    return 3 * x ** 2 - 18 * x

roots = bisection_method(function, derivative, 0.1)
print(f'Raízes encontradas: {roots}')

Intervalo: (-0.6000000000057071, -0.5000000000057071)


,x,f(x),b - a
Iteração,,,
1,-0.550000,0.111125,0.050000
2,-0.575000,-0.165734,0.025000
3,-0.562500,-0.025635,0.012500
4,-0.556250,0.043162,0.006250
5,-0.559375,0.008868,0.003125
6,-0.560938,-0.008357,0.001563
7,-0.560156,0.000262,0.000781



Intervalo: (0.4999999999942304, 0.5999999999942304)


,x,f(x),b - a
Iteração,,,
1,0.550000,0.443875,0.050000
2,0.575000,0.214484,0.025000
3,0.587500,0.096373,0.012500
4,0.593750,0.036469,0.006250
5,0.596875,0.006305,0.003125
6,0.598437,-0.008830,0.001562
7,0.597656,-0.001258,0.000781



Intervalo: (8.899999999993753, 8.999999999993753)


,x,f(x),b - a
Iteração,,,
1,8.950000,-1.005125,0.050000
2,8.975000,0.986234,0.025000
3,8.962500,-0.012240,0.012500
4,8.968750,0.486298,0.006250
5,8.965625,0.236854,0.003125
6,8.964062,0.112263,0.001563
7,8.963281,0.050001,0.000781



Raízes encontradas: [-0.560546875005707, 0.5972656249942304, 8.962890624993753]


## Problema 2

A sequência binária $00111110010100000000000000000000$ (32 bits) pode ser transformada em um número de ponto flutuante no padrão IEEE 754 de precisão simples através do seguinte processo:

- O primeiro bit ($0$) é o bit de sinal. Se ele for $0$, o número é positivo (sinal é igual a $1$), se for $1$ o número é negativo (sinal é igual a $-1$).

- Do segundo ao nono bit ($01111100$) é o número binário que representa o expoente armazenado. O expoente real é calculado pela diferença entre o expoente armazenado e o bias, onde o bias é calculado através da fórmula $2^{k-1} - 1$, onde $k$ é a quantidade de bits reservada para o expoente ($k = 8$ para precisão simples), logo o bias é igual a $127$.

- Do décimo bit em diante ($10100000000000000000000$) estão os bits da mantissa (parte fracionária do número binário).

A parte inteira da mantissa é igual a $1$, logo a conversão do número binário fracionário começa com $1$ e, em seguida, soma-se cada bit da mantissa multiplicado por $2^{-i}$, onde $i$ é a posição desse bit na mantissa. A conversão final é dada pelo sinal multiplicado pela mantissa multiplicado por $2$ elevado ao expoente real.


In [5]:
binary_sequence = '00111110010100000000000000000000'

def ieee754(binary_sequence):
    sign_bit = binary_sequence[0]
    exponent_bits = binary_sequence[1:9]
    mantissa_bits = binary_sequence[9:]

    if sign_bit == '1':
        sign = -1
    else:
        sign = 1

    exponent = binary_to_decimal(exponent_bits) - 127

    mantissa = 1

    for index, bit in enumerate(mantissa_bits, start=1):
        mantissa += int(bit) * (2 ** -index)

    return sign * mantissa * (2 ** exponent)

In [6]:
decimal = ieee754(binary_sequence)
print(f'({binary_sequence})_2 = ({decimal})_10')

(00111110010100000000000000000000)_2 = (0.203125)_10


## Programa 3

Considere um número binário entre 0 e 1 $n = (0.a_0a_1\dots a_{j - 1}a_j\dots)_2$, a sua representação decimal $(0.b_0b_1\dots b_{j - 1}b_j\dots)_{10}$ é obtida através do processo:
1. $k = 0$, $n_k = n$
2. Calcule $w_k = (1010)_2 \cdot n_k$. 
3. Seja $z_k$ a parte inteira de $w_k$.
4. $b_k$ é a conversão de $z_k$ para a base 10.
5. Faça $n_{k +1} = w_k - z_k$.
6. Se $n_{k + 1} = 0$, pare. Caso contrário, faça $k = k + 1$ e repita a partir do passo 2.

Por exemplo, o número binário $(0.000111)_2$ é igual ao número decimal $(0.109375)_{10}$. Podemos visualizar essa transformação através do código abaixo.

In [7]:
def binary_fraction_to_decimal(binary_fraction):
    remainder = binary_fraction
    decimal = 0
    position = -1

    while remainder != '0':
        product = binary_multiply('1010', remainder)

        if '.' in product:
            integer, remainder = product.split('.')
            digit = binary_to_decimal(integer)
            decimal += digit * (10 ** position)

            if '.' not in remainder:
                remainder = '0.' + remainder
        else:
            digit = binary_to_decimal(product)
            decimal += digit * (10 ** position)
            remainder = '0'

        position -= 1

    return decimal

In [8]:
binary = '0.000111'
decimal = binary_fraction_to_decimal(binary)
print(f'({binary})_2 = ({decimal})_10')

(0.000111)_2 = (0.10937500000000001)_10
